In [ ]:
import numpy as np
import pandas as pd

data = pd.read_csv("3DailyReturns.csv", index_col=0, usecols=range(1, 9))

OSError: [Errno 22] Invalid argument: '3DailyReturns.csv'

In [ ]:
# ----------- Parameters -----------
LAMBDA = 1.0
ETA = 0.1
ROWS = 63
TARGET = 0.3221
RISK_FREE_RATE = 0.0472

# ----------- Load and Split Data -----------
data = pd.read_csv("3DailyReturns.csv", index_col=0, usecols=range(1, 9)).dropna()
quarters = [data.iloc[i*ROWS:(i+1)*ROWS] for i in range(4)]

# ----------- Utilities ----------- 
def normalize(w):
    s = np.sum(w)
    return np.ones_like(w) / len(w) if not np.isfinite(s) or s == 0 else w / s

# ----------- Q1: Estimate Mean and Covariance Only -----------
R_q1 = quarters[0].values - 1
mu_q1 = R_q1.mean(axis=0)
cov_q1 = np.cov(R_q1, rowvar=False, ddof=0)

print("\n============== Q1 ESTIMATES (NOT USED IN TRAINING) ==============")
print(f"μ_Q1: {np.round(mu_q1, 6)}")
print(f"Σ_Q1:\n{np.round(cov_q1, 6)}")

# Q2–Q4: Online Training
N = data.shape[1]
w = np.zeros(N)
rl_ret = []
weights_per_q = []
Q2_day1_weights = 0

for q in range(1, 4):  # Q2, Q3, Q4
    q_data = quarters[q]
    daily_returns = []
    w_start = w.copy()  # store Day 1 weights

    for t in range(len(q_data)):
        mu_t = q_data.iloc[t].values - 1
        Sigma_t = np.outer(mu_t, mu_t)
        r_hat = mu_t @ w
        target_daily = TARGET / 3 / ROWS
        tracking_error = r_hat - target_daily
        loss_1 = 0.5 * tracking_error**2
        loss_2 = 0.5 * LAMBDA * (w @ Sigma_t @ w)
        grad = tracking_error * mu_t + LAMBDA * (Sigma_t @ w)
        w = normalize(w - ETA * grad)

        port_ret_t = mu_t @ w
        daily_returns.append(port_ret_t)

    weights_per_q.append(w.copy())
    gross = q_data.prod().values
    G = w @ gross - 1
    rl_ret.append(G)

    daily_returns = np.array(daily_returns)
    q_total = np.prod(1 + daily_returns) - 1
    q_vol = np.std(daily_returns, ddof=1) * np.sqrt(252)
    q_sharpe = (q_total - RISK_FREE_RATE) / q_vol if q_vol != 0 else np.nan

# Final Output 
total_rl = np.prod(1 + np.array(rl_ret)) - 1
vol_rl   = np.std(rl_ret, ddof=1) * np.sqrt(len(rl_ret))
sharpe_rl = (total_rl - RISK_FREE_RATE) / vol_rl if vol_rl != 0 else np.nan

print("\n============== PORTFOLIO WEIGHTS BY QUARTER ==============")
print("Day 1 weights    :", Q2_day1_weight)
for i, w_q in enumerate(weights_per_q, 2):  # Q2–Q4
    print(f"Quarter {i} weights: {np.round(w_q, 4)}")
print("\nFinal weights after Q4:")
print(np.round(w, 4))

print("\n============== RL: GRADIENT DESCENT (Q2–Q4 ONLINE) ===============")
print(f"Scenario return : {total_rl:.4f}")
print(f"Std Dev         : {vol_rl:.4f}")
print(f"Sharpe ratio    : {sharpe_rl:.4f}")

print("\n============== RL-GD-MSE SUMMARY =========================")
print(f"{'Model':<35} {'Return':>10} {'Std Dev':>10} {'Sharpe':>10}")
print("-" * 65)
print(f"{'RL Online (Q2–Q4)':<35} {total_rl:10.4f} {vol_rl:10.4f} {sharpe_rl:10.4f}")
print("-" * 65)


============== Q1 ESTIMATES (NOT USED IN TRAINING) ==============
μ_Q1: [ 0.002653 -0.000657  0.000147 -0.002102  0.006275  0.007983  0.001233]
Σ_Q1:
[[ 2.63e-04  7.30e-05  1.48e-04  6.60e-05  3.66e-04  2.92e-04  1.26e-04]
 [ 7.30e-05  1.18e-04  9.00e-05  3.90e-05  5.90e-05  1.27e-04  5.70e-05]
 [ 1.48e-04  9.00e-05  2.97e-04 -2.00e-05  1.58e-04  1.86e-04  1.14e-04]
 [ 6.60e-05  3.90e-05 -2.00e-05  7.16e-04  1.10e-04  1.15e-04  4.20e-05]
 [ 3.66e-04  5.90e-05  1.58e-04  1.10e-04  8.79e-04  3.52e-04  1.71e-04]
 [ 2.92e-04  1.27e-04  1.86e-04  1.15e-04  3.52e-04  8.33e-04  1.80e-04]
 [ 1.26e-04  5.70e-05  1.14e-04  4.20e-05  1.71e-04  1.80e-04  1.13e-04]]

============== PORTFOLIO WEIGHTS BY QUARTER ==============
Day 1 weights    : [ 0.2687 -0.0477  0.1971 -0.0102  0.1632  0.2413  0.1877]
Quarter 2 weights: [ 0.2704 -0.049   0.1976 -0.0099  0.1626  0.2397  0.1886]
Quarter 3 weights: [ 0.2743 -0.0521  0.2003 -0.0159  0.1633  0.2387  0.1913]
Quarter 4 weights: [ 0.2761 -0.054   0.2016 -0

In [ ]:
# ----------- Parameters -----------
LAMBDA = 1.0
ETA = 0.1
ROWS = 63
TARGET = 0.3221
RISK_FREE_RATE = 0.0472

# ----------- Load and Split Data -----------
data = pd.read_csv("3DailyReturns.csv", index_col=0, usecols=range(1, 9)).dropna()
quarters = [data.iloc[i*ROWS:(i+1)*ROWS] for i in range(4)]

# ----------- Utilities ----------- 
def normalize(w):
    s = np.sum(w)
    return np.ones_like(w) / len(w) if not np.isfinite(s) or s == 0 else w / s

# ----------- Q1: Estimate Mean and Covariance Only -----------
R_q1 = quarters[0].values - 1
mu_q1 = R_q1.mean(axis=0) / ROWS
cov_q1 = np.cov(R_q1, rowvar=False, ddof=0) / ROWS

print("\n============== Q1 ESTIMATES (USED IN Q2) ==============")
print(f"μ_Q1_daily: {np.round(mu_q1, 6)}")
print(f"Σ_Q1_daily:\n{np.round(cov_q1, 6)}")

# ----------- Q2–Q4: Online Training Using Previous Quarter Estimates -----------
N = data.shape[1]
w = np.zeros(N)
rl_ret = []
weights_per_q = []
Q2_day1_weights = 0

for q in range(1, 4):  # Q2, Q3, Q4
    q_data = quarters[q]
    R_prev = quarters[q-1].values - 1
    mu_prev = R_prev.mean(axis=0) / ROWS
    cov_prev = np.cov(R_prev, rowvar=False, ddof=0) / ROWS

    daily_returns = []
    w_start = w.copy()

    for t in range(len(q_data)):
        r_hat = mu_prev @ w
        target_daily = TARGET / 3 / ROWS
        tracking_error = r_hat - target_daily
        loss_1 = 0.5 * tracking_error**2
        loss_2 = 0.5 * LAMBDA * (w @ cov_prev @ w)
        grad = tracking_error * mu_prev + LAMBDA * (cov_prev @ w)
        w = normalize(w - ETA * grad)

        mu_t = q_data.iloc[t].values - 1
        port_ret_t = mu_t @ w
        daily_returns.append(port_ret_t)

        if q == 1:  # Only print daily stats for Q2
            print(f"\nQuarter {q+1} — Day {t+1}")
            print(f"(1/2)(μᵀw - r*)²: {loss_1:.6f}")
            print(f"(λ/2)·variance:  {loss_2:.6f}")
            print(f"Loss:            {loss_1 + loss_2:.6f}")
            print(f"Weights:         {np.round(w, 4)}")
            if t == 0:
                Q2_day1_weight = np.round(w, 4)

    weights_per_q.append(w.copy())
    gross = q_data.prod().values
    G = w @ gross - 1
    rl_ret.append(G)

    daily_returns = np.array(daily_returns)
    q_total = np.prod(1 + daily_returns) - 1
    q_vol = np.std(daily_returns, ddof=1) * np.sqrt(252)
    q_sharpe = (q_total - RISK_FREE_RATE) / q_vol if q_vol != 0 else np.nan

    print(f"\n============== END OF QUARTER {q+1} STATS ==============")
    print(f"Quarter return            : {q_total:.4f}")
    print(f"Annualised Std Dev        : {q_vol:.4f}")
    print(f"Sharpe ratio              : {q_sharpe:.4f}")
    print(f"Weights at Day 1          : {np.round(w_start, 4)}")
    print(f"Weights at Quarter End    : {np.round(w, 4)}")

# ----------- Final Output ----------- 
print(rl_ret)
total_rl = np.prod(1 + np.array(rl_ret)) - 1
vol_rl   = np.std(rl_ret, ddof=1) * np.sqrt(len(rl_ret))
sharpe_rl = (total_rl - RISK_FREE_RATE) / vol_rl if vol_rl != 0 else np.nan

print("\n============== PORTFOLIO WEIGHTS BY QUARTER ==============")
print("Day 1 weights:", Q2_day1_weight)
for i, w_q in enumerate(weights_per_q, 2):  # Q2–Q4
    print(f"Quarter {i} weights: {np.round(w_q, 4)}")
print("\nFinal weights after Q4:")
print(np.round(w, 4))

print("\n============== RL: GRADIENT DESCENT (Q2–Q4 FIXED ESTIMATES) ===============")
print(f"Scenario simple return    : {total_rl:.4f}")
print(f"Annualised Std Dev        : {vol_rl:.4f}")
print(f"Sharpe ratio (rf={RISK_FREE_RATE}) : {sharpe_rl:.4f}")

print("\n============== RL-GD-MSE SUMMARY =========================")
print(f"{'Model':<35} {'Return':>10} {'Std Dev':>10} {'Sharpe':>10}")
print("-" * 65)
print(f"{'RL Fixed μ/Σ per Quarter':<35} {total_rl:10.4f} {vol_rl:10.4f} {sharpe_rl:10.4f}")
print("-" * 65)


============== Q1 ESTIMATES (USED IN Q2) ==============
μ_Q1_daily: [ 4.20e-05 -1.00e-05  2.00e-06 -3.30e-05  1.00e-04  1.27e-04  2.00e-05]
Σ_Q1_daily:
[[ 4.0e-06  1.0e-06  2.0e-06  1.0e-06  6.0e-06  5.0e-06  2.0e-06]
 [ 1.0e-06  2.0e-06  1.0e-06  1.0e-06  1.0e-06  2.0e-06  1.0e-06]
 [ 2.0e-06  1.0e-06  5.0e-06 -0.0e+00  3.0e-06  3.0e-06  2.0e-06]
 [ 1.0e-06  1.0e-06 -0.0e+00  1.1e-05  2.0e-06  2.0e-06  1.0e-06]
 [ 6.0e-06  1.0e-06  3.0e-06  2.0e-06  1.4e-05  6.0e-06  3.0e-06]
 [ 5.0e-06  2.0e-06  3.0e-06  2.0e-06  6.0e-06  1.3e-05  3.0e-06]
 [ 2.0e-06  1.0e-06  2.0e-06  1.0e-06  3.0e-06  3.0e-06  2.0e-06]]

Quarter 2 — Day 1
(1/2)(μᵀw - r*)²: 0.000001
(λ/2)·variance:  0.000000
Loss:            0.000001
Weights:         [ 0.1708 -0.0423  0.0095 -0.1353  0.404   0.5139  0.0794]

Quarter 2 — Day 2
(1/2)(μᵀw - r*)²: 0.000001
(λ/2)·variance:  0.000005
Loss:            0.000006
Weights:         [ 0.1708 -0.0423  0.0095 -0.1353  0.404   0.5139  0.0794]

Quarter 2 — Day 3
(1/2)(μᵀw - r*)²: 0